In [10]:
import os
import pandas as pd

# ======= CONFIGURAÇÕES DE TODAS AS MODALIDADES =======
modalidades = [
    {
        'nome': 'audio_balanced',
        'master': 'metadata/master_metadata_audio_balanced.csv',
        'data_subfolder': 'data/audio/audio_balanced',
        'file_ext': '',      # coloque '.mp3', '.wav' se houver
        'type': 'audio'
    },
    {
        'nome': 'audio_complete',
        'master': 'metadata/master_metadata_audio_complete.csv',
        'data_subfolder': 'data/audio/audio_complete',
        'file_ext': '',
        'type': 'audio'
    },
    {
        'nome': 'lyrics_balanced',
        'master': 'metadata/master_metadata_lyrics_balanced.csv',
        'data_subfolder': 'data/lyrics/lyrics_balanced',
        'file_ext': '.txt',
        'type': 'lyrics'
    },
    {
        'nome': 'lyrics_complete',
        'master': 'metadata/master_metadata_lyrics_complete.csv',
        'data_subfolder': 'data/lyrics/lyrics_complete',
        'file_ext': '.txt',
        'type': 'lyrics'
    },
    {
        'nome': 'bimodal_balanced_audio',
        'master': 'metadata/master_metadata_bimodal_balanced.csv',
        'data_subfolder': 'data/bimodal/bimodal_balanced/audio',
        'file_ext': '',
        'type': 'audio'
    },
    {
        'nome': 'bimodal_balanced_lyrics',
        'master': 'metadata/master_metadata_bimodal_balanced.csv',
        'data_subfolder': 'data/bimodal/bimodal_balanced/lyrics',
        'file_ext': '.txt',
        'type': 'lyrics'
    },
    {
        'nome': 'bimodal_complete_audio',
        'master': 'metadata/master_metadata_bimodal_complete.csv',
        'data_subfolder': 'data/bimodal/bimodal_complete/audio',
        'file_ext': '',
        'type': 'audio'
    },
    {
        'nome': 'bimodal_complete_lyrics',
        'master': 'metadata/master_metadata_bimodal_complete.csv',
        'data_subfolder': 'data/bimodal/bimodal_complete/lyrics',
        'file_ext': '.txt',
        'type': 'lyrics'
    },
]

# ======= FUNÇÃO DE CHECAGEM =======
def checar_modalidade(nome, master_csv, data_subfolder, file_ext, tipo):
    missing = []
    try:
        df = pd.read_csv(master_csv)
    except Exception as e:
        print(f"ERRO ao ler {master_csv}: {e}")
        return []

    print(f"\n[{nome}] Total de músicas no master: {len(df)}")

    for idx, row in df.iterrows():
        quad = None
        for qcol in ['quadrant', 'Quadrant']:
            if qcol in df.columns:
                quad = row[qcol]
                break
        if quad is None:
            print(f"[{nome}] Coluna 'quadrant' não encontrada!")
            break

        song_id = row['song_id'] if 'song_id' in df.columns else row['Song']

        # Monta caminho do arquivo físico
        file_path = os.path.join(data_subfolder, f'Q{quad}', f'{song_id}{file_ext}')
        if not os.path.exists(file_path):
            missing.append({'modalidade': nome, 'song_id': song_id, 'quadrant': quad, 'path_esperado': file_path})

    print(f"[{nome}] Arquivos faltando: {len(missing)}")
    return missing

# ======= EXECUTA PARA TODAS AS MODALIDADES =======
all_missing = []
for m in modalidades:
    faltantes = checar_modalidade(m['nome'], m['master'], m['data_subfolder'], m['file_ext'], m['type'])
    all_missing.extend(faltantes)

# ======= SALVA RELATÓRIO FINAL =======
if all_missing:
    df_missing = pd.DataFrame(all_missing)
    df_missing.to_csv('relatorio_arquivos_faltantes.csv', index=False)
    print("\nRelatório salvo em relatorio_arquivos_faltantes.csv")
else:
    print("\nNenhum arquivo faltando em nenhuma modalidade!")


ERRO ao ler metadata/master_metadata_audio_balanced.csv: [Errno 2] No such file or directory: 'metadata/master_metadata_audio_balanced.csv'
ERRO ao ler metadata/master_metadata_audio_complete.csv: [Errno 2] No such file or directory: 'metadata/master_metadata_audio_complete.csv'
ERRO ao ler metadata/master_metadata_lyrics_balanced.csv: [Errno 2] No such file or directory: 'metadata/master_metadata_lyrics_balanced.csv'
ERRO ao ler metadata/master_metadata_lyrics_complete.csv: [Errno 2] No such file or directory: 'metadata/master_metadata_lyrics_complete.csv'
ERRO ao ler metadata/master_metadata_bimodal_balanced.csv: [Errno 2] No such file or directory: 'metadata/master_metadata_bimodal_balanced.csv'
ERRO ao ler metadata/master_metadata_bimodal_balanced.csv: [Errno 2] No such file or directory: 'metadata/master_metadata_bimodal_balanced.csv'
ERRO ao ler metadata/master_metadata_bimodal_complete.csv: [Errno 2] No such file or directory: 'metadata/master_metadata_bimodal_complete.csv'
ERRO